# Recurrent neural network implementation

This notebook contains a Python implementation of a simple recurrent neural network with a single hidden layer.

In [1]:
import numpy as np

In [2]:
with open("input.txt") as f:
    data = list(f.read())

In [3]:
chars = list(set(data))

In [4]:
char_to_index = {chars[i]: i for i in range(len(chars))}
index_to_chars = {char_to_index[c]: c for c in chars}

In [5]:
num_chars = len(chars)
data_len = len(data)

In [6]:
X = np.zeros((data_len, num_chars))
X[np.arange(data_len), [char_to_index[s] for s in data]] = 1
Y = X[1:,:]
X = X[:-1,:]

In [7]:
training_examples = 10000

X = X[:training_examples]
Y = Y[:training_examples]

In [8]:
print(X.shape)
print(Y.shape)

(10000, 65)
(10000, 65)


In [9]:
sequence_length = 25
hidden_units = 128
learning_rate = 0.001
max_bptt_steps = 5

In [10]:
# propagage a sequence forward through the network and return the outputs and hidden states
def forward_prop(inputs):
    h = np.zeros((len(inputs)+1, hidden_units, 1))
    o = np.zeros((len(inputs), num_chars, 1))

    for t in range(len(inputs)):
        x_t = inputs[t]             
        h[t] = np.tanh(np.dot(U, x_t) + np.dot(W, h[t-1]) + b)
        z_t = np.dot(V, h[t]) + c
        o[t] = np.exp(z_t)/np.sum(np.exp(z_t))
    
    return o, h

In [11]:
def back_prop(inputs, targets):
    
    # initialize gradients
    dU = np.zeros_like(U)
    dW = np.zeros_like(W)
    db = np.zeros_like(b)
    dV = np.zeros_like(V)
    dc = np.zeros_like(c)      
    
    for t in reversed(range(sequence_length)):
        dV += np.outer(o[t] - targets[t], h[t])
        dc += (o[t] - targets[t])
        dz_t = np.dot(V.T, o[t] - targets[t]) * (1 - h[t] ** 2)
        for k in reversed(range(max(0, t - max_bptt_steps), t + 1)):
            dW += np.outer(dz_t, h[k-1])
            dU += np.outer(dz_t, inputs[k])
            db += dz_t
            dz_t = np.dot(W.T, dz_t) * (1 - h[k] ** 2)
    
    return dU, dW, db, dV, dc

In [12]:
def calculate_total_loss(targets, predictions):
    return -1 * np.sum(np.log(predictions[np.arange(len(targets)), np.argmax(targets, axis=1).reshape(-1)]))

def calculate_cost(X, Y):
    L = 0
    N = 0
    
    pos = 0
    while pos <= (len(X)-sequence_length):
        s = slice(pos, (pos+sequence_length))
        inputs = np.expand_dims(X[s,:], axis=-1)
        targets = np.expand_dims(Y[s,:], axis=-1)        
        o, h = forward_prop(inputs)
        L += calculate_total_loss(targets, o)
        N += targets.shape[0]
        pos += sequence_length
        
    return L / N

In [13]:
# initialize parameters
U = np.random.uniform(low=-1/np.sqrt(num_chars), high=1/np.sqrt(num_chars), size=(hidden_units, num_chars))
W = np.random.uniform(low=-1/np.sqrt(hidden_units), high=1/np.sqrt(hidden_units), size=(hidden_units, hidden_units))
b = np.zeros((hidden_units, 1))
V = np.random.uniform(low=-1/np.sqrt(hidden_units), high=1/np.sqrt(hidden_units), size=(num_chars, hidden_units))
c = np.zeros((num_chars, 1))

print(U.shape)
print(W.shape)
print(b.shape)
print(V.shape)
print(c.shape)

for i in range(10):

    print(f"epoch: {i}, cost: {calculate_cost(X, Y)}")

    pos = 0
    while pos <= (X.shape[0]-sequence_length):       

        s = slice(pos, (pos+sequence_length))
        inputs = np.expand_dims(X[s,:], axis=-1)
        targets = np.expand_dims(Y[s,:], axis=-1)  
        
        # forward prop
        o, h = forward_prop(inputs)

        # back prop
        dU, dW, db, dV, dc = back_prop(inputs, targets)
                
        U = U - learning_rate * dU
        W = W - learning_rate * dW
        b = b - learning_rate * db
        V = V - learning_rate * dV
        c = c - learning_rate * dc                 

        pos += sequence_length


(128, 65)
(128, 128)
(128, 1)
(65, 128)
(65, 1)
epoch: 0, cost: 4.178889636225676
epoch: 1, cost: 3.262298094106931
epoch: 2, cost: 3.2061573133875116
epoch: 3, cost: 3.1304089179587185
epoch: 4, cost: 3.020501821352576
epoch: 5, cost: 2.9241922110315297
epoch: 6, cost: 2.8428504187759285
epoch: 7, cost: 2.7720839006548728
epoch: 8, cost: 2.7085786748051945
epoch: 9, cost: 2.650344119506688
